In [3]:
import socket
import time
import numpy as np
import hmac

def divide_text(text, block_size_bits=16, X=None):
    text = text.encode('utf-8')
    block_size_bytes = block_size_bits // 8
    blocks = [text[i:i + block_size_bytes] for i in range(0, len(text), block_size_bytes)]
    
    # Padding to ensure the number of blocks is a multiple of X.shape[0]
    if X is not None and len(blocks) % X.shape[0] != 0:
        padding_size = X.shape[0] - len(blocks) % X.shape[0]
        blocks.extend([b''] * padding_size)
    
    return np.array(blocks)

def mac_for_block(blocks, key, X, Y):
    res = {}
    for msg_index in range(X.shape[0]):
        res[msg_index] = blocks[msg_index]
    
    for tag_index in range(X.shape[1]):
        selected_blocks = blocks[X[:, tag_index] == 1]
        if selected_blocks.size > 0:
            data = b''.join(selected_blocks)
            target_index = np.where(Y[:, tag_index] == 1)[0][0]
            res[target_index] = blocks[target_index] + hmac.new(key, data, digestmod='sha384').digest()
    
    return res

def send_msg(FN, msg, sock, dest):
    sock.sendto(FN.to_bytes(4, 'big') + msg, dest)

def tx(text, sock, ip, port, X, Y, block_size_bits=128, attack=[]):
    blocks = divide_text(text, block_size_bits=block_size_bits, X=X)
    FN = 1
    total_mac_length = 0
    
    for i in range(0, len(blocks), X.shape[0]):
        block = mac_for_block(blocks[i:i + X.shape[0]], key=b"key", X=X, Y=Y)
        total_mac_length += X.shape[1] * 384 // 8
        
        for msg in block.values():
            if FN in attack:
                FN += 1
                continue
            send_msg(FN, msg, sock, (ip, port))
            FN += 1
            time.sleep(0.01)
    
    sock.sendto(b'END', (ip, port))
    sock.close()
    time.sleep(1)
    return total_mac_length

# Matrix Definitions (X and Y)
X = np.array([[1, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
              [1, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
              [1, 0, 0, 0, 0, 0, 0, 0, 1],  # m3
              [0, 1, 0, 0, 0, 0, 1, 0, 0],  # m4
              [0, 1, 0, 0, 0, 0, 0, 1, 0],  # m5
              [0, 1, 0, 0, 0, 0, 0, 0, 1],  # m6
              [0, 0, 1, 0, 0, 0, 1, 0, 0],  # m7
              [0, 0, 1, 0, 0, 0, 0, 1, 0],  # m8
              [0, 0, 1, 0, 0, 0, 0, 0, 1],  # m9
              [0, 0, 0, 1, 0, 0, 1, 0, 0],  # m10
              [0, 0, 0, 1, 0, 0, 0, 1, 0],  # m11
              [0, 0, 0, 1, 0, 0, 0, 0, 1],  # m12
              [0, 0, 0, 0, 1, 0, 1, 0, 0],  # m13
              [0, 0, 0, 0, 1, 0, 0, 1, 0],  # m14
              [0, 0, 0, 0, 1, 0, 0, 0, 1],  # m15
              [0, 0, 0, 0, 0, 1, 1, 0, 0],  # m16
              [0, 0, 0, 0, 0, 1, 0, 1, 0],  # m17
              [0, 0, 0, 0, 0, 1, 0, 0, 1]]) # m18

Y = np.array([[0, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
              [0, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
              [1, 0, 0, 0, 0, 0, 0, 0, 0],  # m3
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m4
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m5
              [0, 1, 0, 0, 0, 0, 0, 0, 0],  # m6
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m7
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m8
              [0, 0, 1, 0, 0, 0, 0, 0, 0],  # m9
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m10
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m11
              [0, 0, 0, 1, 0, 0, 0, 0, 0],  # m12
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m13
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m14
              [0, 0, 0, 0, 1, 0, 0, 0, 0],  # m15
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m16
              [0, 0, 0, 0, 0, 1, 0, 0, 0],  # m17
              [0, 0, 0, 0, 0, 0, 0, 0, 1]]) # m18

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ip = "10.42.0.15"
port = 23422

message_size_in_bits = 1024

with open("send.txt", "r") as f:
    text = f.read()

attack = np.random.randint(1, len(divide_text(text, message_size_in_bits, X)), 0)
print(f'Attacking on packet numbers {attack}\nTransmitting text: {text}\n\n')

tx(text, sock, ip, port, X, Y, block_size_bits=message_size_in_bits, attack=attack)


Attacking on packet numbers []
Transmitting text: This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.
This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.
This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test shows  2D integrity check is better than  blockwise integrity.This test sh

44496